In [ ]:
# !pip install requests==2.32.3 geopandas==1.0.1
# import requests
# import geopandas as gpd
# print(requests.__version__)
# print(gpd.__version__)

In [ ]:
import requests
import geopandas as gpd
from shapely.geometry import Point

# OSM amenity
- https://wiki.openstreetmap.org/wiki/Key:amenity
- https://wiki.openstreetmap.org/wiki/Key:shop
- 전체적인 tag 확인
  - https://taginfo.openstreetmap.org/keys/highway#values
  - https://wiki.openstreetmap.org/wiki/Ko:%ED%83%9C%EA%B7%B8
  
## ✅ OSM에서 자주 쓰이는 주요 Key 목록

### 1️⃣ **장소 및 시설 관련 (`amenity`, `shop`, `office` 등)**

| Key | 설명 | 예시 |
| --- | --- | --- |
| `amenity` | 공공시설 및 편의시설 | `amenity=restaurant` (음식점) |
| `shop` | 상점 및 가게 | `shop=convenience` (편의점) |
| `office` | 사무실 유형 | `office=it` (IT 회사) |
| `tourism` | 관광 관련 시설 | `tourism=hotel` (호텔) |

---

### 2️⃣ **교통 및 도로 (`highway`, `railway`, `public_transport` 등)**

| Key | 설명 | 예시 |
| --- | --- | --- |
| `highway` | 도로 유형 | `highway=motorway` (고속도로) |
| `railway` | 철도 및 지하철 | `railway=subway` (지하철) |
| `public_transport` | 대중교통 시설 | `public_transport=station` (기차역, 지하철역) |
| `aeroway` | 공항 관련 시설 | `aeroway=runway` (활주로) |

---

### 3️⃣ **건물 및 주소 (`building`, `addr:*` 등)**

| Key | 설명 | 예시 |
| --- | --- | --- |
| `building` | 건물 유형 | `building=apartments` (아파트) |
| `addr:housenumber` | 건물 번호 | `addr:housenumber=123` |
| `addr:street` | 도로명 주소 | `addr:street=Gangnam-daero` |
| `addr:city` | 도시명 | `addr:city=Seoul` |

---

### 4️⃣ **지형 및 자연 (`natural`, `landuse`, `waterway` 등)**

| Key | 설명 | 예시 |
| --- | --- | --- |
| `natural` | 자연 지형 | `natural=tree` (나무) |
| `landuse` | 토지 이용 목적 | `landuse=residential` (주거 지역) |
| `waterway` | 수로 및 하천 | `waterway=river` (강) |

---

### 5️⃣ **기타 유용한 태그**

| Key | 설명 | 예시 |
| --- | --- | --- |
| `name` | 이름 | `name=GS25 강남역점` |
| `brand` | 브랜드명 | `brand=7-Eleven` |
| `operator` | 운영 주체 | `operator=BGF Retail` |
| `opening_hours` | 영업 시간 | `opening_hours=24/7` |
| `website` | 공식 웹사이트 | `website=https://www.7-eleven.co.kr/` |
| `phone` | 전화번호 | `phone=+82-2-1234-5678` |


## 공공시설 및 편의시설, 지하철, 상점/가게

### 공공시설

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"

overpass_query = """
[out:json];
area(3602297418)->.searchArea;  // 서울의 Area ID
(
  node["amenity"](area.searchArea);
);
out body;
"""
# Overpass API 요청 보내기
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# 결과 확인
data['elements'][13]


pois= []
for element in data['elements']:
    # print(element)
    lon = element.get('lon', None)
    lat = element.get('lat', None)
    tags = element.get('tags', None)
    if lon and lat and tags:
        pois.append({
            # 'highway': tags.get('highway', None),
            'geometry': Point(float(lon), float(lat)),
            'name': tags.get('name', tags.get("name.ko", tags.get('description', tags.get('name:en', None)))),
            'item': tags.get('amenity', None),
            'lon' : lon,
            'lat' : lat
        })

pois_gdf = gpd.GeoDataFrame(pois, columns=['name', 'item' ,'lon', 'lat','geometry'])
pois_gdf

### 대중교통 지하철

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"

overpass_query = """
[out:json];
area(3602297418)->.searchArea;  // 서울의 Area ID
(
  node["public_transport"]["subway"="yes"]["public_transport"="station"](area.searchArea);
  node["highway"="bus_stop"](area.searchArea);
);
out body;
"""

# Overpass API 요청 보내기
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# 결과 확인
data

pois= []
for element in data['elements']:
    # print(element)
    lon = element.get('lon', None)
    lat = element.get('lat', None)
    tags = element.get('tags', None)
    if lon and lat and tags:
        pois.append({
            # 'highway': tags.get('highway', None),
            'geometry': Point(float(lon), float(lat)),
            'name': tags.get('name', None),
            'item': tags.get('station', tags.get('highway', None)),
            'lon' : lon,
            'lat' : lat
        })

pois_gdf_trans = gpd.GeoDataFrame(pois, columns=['name', 'item' ,'lon', 'lat','geometry'])
pois_gdf_trans

### Shop

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"

overpass_query = """
[out:json];
area(3602297418)->.searchArea;  // 서울의 Area ID
(
  node["shop"](area.searchArea);
  way["shop"](area.searchArea);
  relation["shop"](area.searchArea);
);
out center;
"""

# Overpass API 요청 보내기
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# 결과 확인
data

pois= []
for element in data['elements']:
    # print(element)
    lon = element.get('lon', None)
    lat = element.get('lat', None)
    tags = element.get('tags', None)
    if lon and lat and tags:
        pois.append({
            # 'highway': tags.get('highway', None),
            'geometry': Point(float(lon), float(lat)),
            'name': tags.get('name', None),
            'item': f"shop_{tags.get('shop', None)}",
            'lon' : lon,
            'lat' : lat
        })

pois_gdf_shop = gpd.GeoDataFrame(pois, columns=['name', 'item' ,'lon', 'lat','geometry'])
pois_gdf_shop

In [ ]:
print(len(pois_gdf), len(pois_gdf_trans), len(pois_gdf_shop))
pois_gdf_dropna=pois_gdf.dropna(subset=['name'], inplace=False)
pois_gdf_trans_dropna=pois_gdf_trans.dropna(subset=['name'], inplace=False)
pois_gdf_shop_dropna=pois_gdf_shop.dropna(subset=['name'], inplace=False)
print(len(pois_gdf_dropna), len(pois_gdf_trans_dropna), len(pois_gdf_shop_dropna))
print("Total data length : ",len(pois_gdf_dropna)+ len(pois_gdf_trans_dropna)+ len(pois_gdf_shop_dropna))

In [ ]:
df_poi=pd.concat([pois_gdf_dropna, pois_gdf_trans_dropna, pois_gdf_shop_dropna], axis=0)
df_poi

### 데이터 저장

In [ ]:
import sqlalchemy
import pandas as pd
from sqlalchemy import create_engine
import geopandas as gpd

import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

hostname = os.getenv('db_hostname')
username = os.getenv('db_username')
password = os.getenv('db_password')
port = os.getenv('db_port')
database= os.getenv('database')

connect_url = f"postgresql://{username}:{password}@{hostname}:{port}/{database}"
engine = create_engine(connect_url)

# 데이터베이스 저장
poi_gdf=gpd.GeoDataFrame(df_poi, geometry = 'geometry')
poi_gdf = poi_gdf.set_crs('EPSG:4326', allow_override=True) # epsg 4326
# gdf_sgg_4326=emd_cell_crs.to_crs(epsg=4326)
poi_gdf.to_postgis(name='tb_poi', con=engine, if_exists='replace')

# 파일로 저장
poi_gdf.to_file("tb_poi.geojson", driver='GeoJSON')
poi_gdf.to_csv("tb_poi.csv", index=False)
pd.DataFrame(df_poi.item.value_counts()).to_csv("poi_items.csv")


# 검토 - 각 key:tag 별 데이터 확인용

### 버스정류장

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"

overpass_query = """
[out:json];
area(3602297418)->.searchArea;  // 서울의 Area ID
(
  node["highway"="bus_stop"](area.searchArea);
  way["highway"="bus_stop"](area.searchArea);
  relation["highway"="bus_stop"](area.searchArea);
  
  node["amenity"="bus_station"](area.searchArea);
  way["amenity"="bus_station"](area.searchArea); 
  
  // 음식점, 카페 등등
  node["amenity"="restaurant"](area.searchArea);
  way["amenity"="restaurant"](area.searchArea);
  relation["amenity"="restaurant"](area.searchArea);
  
  node["amenity"="fast_food"](area.searchArea);
  way["amenity"="fast_food"](area.searchArea);
  relation["amenity"="fast_food"](area.searchArea);
  
  node["amenity"="cafe"](area.searchArea);
  way["amenity"="cafe"](area.searchArea);
  relation["amenity"="cafe"](area.searchArea);
  
  node["amenity"="hospital"](area.searchArea);
  way["amenity"="hospital"](area.searchArea);
  relation["amenity"="hospital"](area.searchArea);

  node["amenity"="dentist"](area.searchArea);
  way["amenity"="dentist"](area.searchArea);
  relation["amenity"="dentist"](area.searchArea);
  
  node["amenity"="clinic"](area.searchArea);
  way["amenity"="clinic"](area.searchArea);
  relation["amenity"="clinic"](area.searchArea);

  node["amenity"="doctors"](area.searchArea);
  way["amenity"="doctors"](area.searchArea);
  relation["amenity"="doctors"](area.searchArea);

  node["amenity"="pharmacy"](area.searchArea);
  way["amenity"="pharmacy"](area.searchArea);
  relation["amenity"="pharmacy"](area.searchArea);

  node["amenity"="veterinary"](area.searchArea);
  way["amenity"="veterinary"](area.searchArea);
  relation["amenity"="veterinary"](area.searchArea);

  node["amenity"="college"](area.searchArea);
  way["amenity"="college"](area.searchArea);
  relation["amenity"="college"](area.searchArea);
  
  node["amenity"="school"](area.searchArea);
  way["amenity"="school"](area.searchArea);
  relation["amenity"="school"](area.searchArea);
  
  node["amenity"="university"](area.searchArea);
  way["amenity"="university"](area.searchArea);
  relation["amenity"="university"](area.searchArea);
);
out body;
"""

# Overpass API 요청 보내기
response = requests.get(overpass_url, params={'data': overpass_query})
print(response)
data = response.json()

# 결과 확인
data

pois= []
for element in data['elements']:
    # print(element)
    lon = element.get('lon', None)
    lat = element.get('lat', None)
    tags = element.get('tags', None)
    if lon and lat and tags:
        pois.append({
            # 'highway': tags.get('highway', None),
            'geometry': Point(float(lon), float(lat)),
            'name': tags.get('name', None),
            'amenity': tags.get('amenity', tags.get('highway', None)),
            'lon' : lon,
            'lat' : lat
        })

pois_gdf = gpd.GeoDataFrame(pois, columns=['geometry', 'name', 'amenity', 'lon', 'lat'])
pois_gdf

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"

overpass_query = """
[out:json];
area(3602297418)->.searchArea;  // 서울의 Area ID
(
  node["highway"="*"](area.searchArea);
  way["highway"="bus_stop"](area.searchArea);
  relation["highway"="bus_stop"](area.searchArea);
  
  node["amenity"="bus_station"](area.searchArea);
  way["amenity"="bus_station"](area.searchArea); 
);
out body;
"""

# Overpass API 요청 보내기
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# 결과 확인
data

pois= []
for element in data['elements']:
    # print(element)
    lon = element.get('lon', None)
    lat = element.get('lat', None)
    tags = element.get('tags', None)
    if lon and lat and tags:
        pois.append({
            # 'highway': tags.get('highway', None),
            'geometry': Point(float(lon), float(lat)),
            'name': tags.get('name', None),
            'amenity': tags.get('amenity', tags.get('highway', None)),
            'lon' : lon,
            'lat' : lat
        })

pois_gdf = gpd.GeoDataFrame(pois, columns=['geometry', 'name', 'amenity', 'lon', 'lat'])
pois_gdf

### restaurant

In [ ]:
### 음식점
overpass_url = "http://overpass-api.de/api/interpreter"

overpass_query = """
[out:json];
area(3602297418)->.searchArea;  // 서울의 Area ID
(
  node["amenity"="restaurant"](area.searchArea);
  way["amenity"="restaurant"](area.searchArea);
  relation["amenity"="restaurant"](area.searchArea);
);
out body;
"""

# Overpass API 요청 보내기
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# 결과 확인
data

pois= []
for element in data['elements']:
    # print(element)
    lon = element.get('lon', None)
    lat = element.get('lat', None)
    tags = element.get('tags', None)
    if lon and lat and tags:
        pois.append({
            # 'highway': tags.get('highway', None),
            'geometry': Point(float(lon), float(lat)),
            'name': tags.get('name', None),
            'amenity': tags.get('amenity', tags.get('highway', None)),
            'lon' : lon,
            'lat' : lat
        })

pois_gdf = gpd.GeoDataFrame(pois, columns=['geometry', 'name', 'amenity', 'lon', 'lat'])
pois_gdf

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"

overpass_query = """
[out:json];
area(3602297418)->.searchArea;  // 서울의 Area ID
(
  node["railway"="station"](area.searchArea);
  node["railway"="subway_entrance"](area.searchArea);
);
out body;
"""
# Overpass API 요청 보내기
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# 결과 확인
data


### 병원, 클리닉, 치과, 약국 등

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"

overpass_query = """
[out:json][timeout:900];
area(3602297418)->.searchArea;  // 서울의 Area ID
node(area.searchArea);
out body 100000;
"""

# Overpass API 요청 보내기
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# 결과 확인
data

pois= []
for element in data['elements']:
    # print(element)
    lon = element.get('lon', None)
    lat = element.get('lat', None)
    tags = element.get('tags', None)
    if lon and lat and tags:
        pois.append({
            # 'highway': tags.get('highway', None),
            'geometry': Point(float(lon), float(lat)),
            'name': tags.get('name', None),
            'amenity': tags.get('amenity', tags.get('highway', None)),
            'lon' : lon,
            'lat' : lat
        })

pois_gdf = gpd.GeoDataFrame(pois, columns=['geometry', 'name', 'amenity', 'lon', 'lat'])
pois_gdf

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"

overpass_query = """
[out:json];
area(3602297418)->.searchArea;  // 서울의 Area ID
(
  node["amenity"="hospital"](area.searchArea);
  way["amenity"="hospital"](area.searchArea);
  relation["amenity"="hospital"](area.searchArea);

  node["amenity"="dentist"](area.searchArea);
  way["amenity"="dentist"](area.searchArea);
  relation["amenity"="dentist"](area.searchArea);
  
  node["amenity"="clinic"](area.searchArea);
  way["amenity"="clinic"](area.searchArea);
  relation["amenity"="clinic"](area.searchArea);

  node["amenity"="doctors"](area.searchArea);
  way["amenity"="doctors"](area.searchArea);
  relation["amenity"="doctors"](area.searchArea);

  node["amenity"="pharmacy"](area.searchArea);
  way["amenity"="pharmacy"](area.searchArea);
  relation["amenity"="pharmacy"](area.searchArea);

  node["amenity"="veterinary"](area.searchArea);
  way["amenity"="veterinary"](area.searchArea);
  relation["amenity"="veterinary"](area.searchArea);

);
out body;
"""

# Overpass API 요청 보내기
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# 결과 확인
data

pois= []
for element in data['elements']:
    # print(element)
    lon = element.get('lon', None)
    lat = element.get('lat', None)
    tags = element.get('tags', None)
    if lon and lat and tags:
        pois.append({
            # 'highway': tags.get('highway', None),
            'geometry': Point(float(lon), float(lat)),
            'name': tags.get('name', None),
            'amenity': tags.get('amenity', tags.get('highway', None)),
            'lon' : lon,
            'lat' : lat
        })

pois_gdf = gpd.GeoDataFrame(pois, columns=['geometry', 'name', 'amenity', 'lon', 'lat'])
pois_gdf

### 학교

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"

overpass_query = """
[out:json];
area(3602297418)->.searchArea;  // 서울의 Area ID
(
  node["amenity"="college"](area.searchArea);
  way["amenity"="college"](area.searchArea);
  relation["amenity"="college"](area.searchArea);
  
  node["amenity"="school"](area.searchArea);
  way["amenity"="school"](area.searchArea);
  relation["amenity"="school"](area.searchArea);
  
  node["amenity"="university"](area.searchArea);
  way["amenity"="university"](area.searchArea);
  relation["amenity"="university"](area.searchArea);
);
out body;
"""

# Overpass API 요청 보내기
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# 결과 확인
data

pois= []
for element in data['elements']:
    # print(element)
    lon = element.get('lon', None)
    lat = element.get('lat', None)
    tags = element.get('tags', None)
    if lon and lat and tags:
        pois.append({
            # 'highway': tags.get('highway', None),
            'geometry': Point(float(lon), float(lat)),
            'name': tags.get('name', None),
            'amenity': tags.get('amenity', tags.get('highway', None)),
            'lon' : lon,
            'lat' : lat
        })

pois_gdf = gpd.GeoDataFrame(pois, columns=['geometry', 'name', 'amenity', 'lon', 'lat'])
pois_gdf

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"

overpass_query = """
[out:json];
area(3602297418)->.searchArea; 
// 시설 검색
(
  node["amenity"="atm"](area.searchArea);
  way["amenity"="atm"](area.searchArea);
  relation["amenity"="atm"](area.searchArea);
  
  node["amenity"="bank"](area.searchArea);
  way["amenity"="bank"](area.searchArea);
  relation["amenity"="bank"](area.searchArea);
);
// 결과 출력
out body;

"""

# Overpass API 요청 보내기
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# 결과 확인
data

pois= []
for element in data['elements']:
    # print(element)
    lon = element.get('lon', None)
    lat = element.get('lat', None)
    tags = element.get('tags', None)
    if lon and lat and tags:
        pois.append({
            'geometry': Point(float(lon), float(lat)),
            'name': tags.get('name', None),
            'amenity': tags.get('amenity', None),
            'lon' : lon,
            'lat' : lat
        })

pois_gdf = gpd.GeoDataFrame(pois, columns=['geometry', 'name', 'amenity', 'lon', 'lat'])
pois_gdf

In [ ]:
data

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"

overpass_query = """
[out:json];
area(3602297418)->.searchArea; 
// 시설 검색
(
  node["railway"="subway_entrance"](area.searchArea);
  way["railway"="station"]["station"="subway"](area.searchArea);
  relation["railway"="station"]["station"="subway"](area.searchArea);
);
// 결과 출력
out body;

"""

# Overpass API 요청 보내기
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()


pois= []
for element in data['elements']:
    # print(element)
    lon = element.get('lon', None)
    lat = element.get('lat', None)
    tags = element.get('tags', None)
    if lon and lat and tags:
        pois.append({
            'geometry': Point(float(lon), float(lat)),
            'name': tags.get('name', tags.get('description', tags.get('description:en',None))),
            'amenity': tags.get('amenity', tags.get('railway', None)),
            'lon' : lon,
            'lat' : lat
        })

pois_gdf = gpd.GeoDataFrame(pois, columns=['geometry', 'name', 'amenity', 'lon', 'lat'])
pois_gdf

In [ ]:
pois_gdf['name'].isnull().sum()

In [ ]:
nodes=data['elements']
nodes[1828]

In [ ]:
tags_list =  [node['tags'] for node in data['elements'] if 'tags' in node]
lists = [list(item.keys()) for item in tags_list]
lists
# unique_keys = set(key for item in tags_list for key in item.keys())

# # 고유한 키 목록 출력
# print(unique_keys)

In [ ]:
# 각 리스트의 항목을 집합으로 변환하여 중복 항목 계산
all_elements = [item for sublist in lists for item in sublist]
unique_elements = set(item for item in all_elements if all_elements.count(item) == 1)

# 중복되지 않는 항목만 포함하는 리스트 필터링
filtered_lists = []
for sublist in lists:
    filtered_lists.append([item for item in sublist if item in unique_elements])

# 결과 출력
print(filtered_lists)

In [ ]:
import requests

# Overpass API URL
overpass_url = "http://overpass-api.de/api/interpreter"

# Area ID로 OSM 데이터를 가져오는 쿼리
overpass_query = """
[out:json];
area(162177234);
out body;
"""

# Overpass API 요청 보내기
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# 결과 확인
print(data)


In [ ]:
import requests

# Overpass API URL
overpass_url = "http://overpass-api.de/api/interpreter"

# Area ID로 OSM 데이터를 가져오는 쿼리
overpass_query = """
[out:json];
area(2297418);
out body;
"""

# Overpass API 요청 보내기
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# 결과 확인
print(data)